# Detecção de falhas em equipamentos por meio de inspeção visual

Iniciação científica voluntária

Murilo Capozzi dos Santos

Orientadora: Profa. Dra. Lilian Berton

Instituto de Ciência e Tecnologia - UNIFESP

murilo.capozzi@unifesp.br



---



In [ ]:
import tensorflow as tf
import keras

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
import keras.utils as image
from keras.utils import load_img
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

import cv2
from google.colab.patches import cv2_imshow

import time
import copy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.efficientnet import EfficientNetB0
from keras.api._v2.keras import Model

from IPython.display import Image, display
import matplotlib.cm as cm

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
def grafico(model_history, model_name):

  history = model_history

  acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()),1])
  plt.title(model_name + ' - Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.ylabel('Cross Entropy')
  plt.ylim([0,1.0])
  plt.title(model_name + ' - Training and Validation Loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
def evaluate(model, base, model_name):
  pred = model.predict(base)

  test_loss, test_accuracy = model.evaluate(base, batch_size=batch_size)

  print()
  print(f"Test Loss:     {test_loss}")
  print(f"Test Accuracy: {test_accuracy}")

  y_pred = np.argmax(pred, axis=1)
  y_true = base.classes

  print(classification_report(y_true, y_pred))

  cf_mtx = confusion_matrix(y_true, y_pred)

  group_counts = ["{0:0.0f}".format(value) for value in cf_mtx.flatten()]
  group_percentages = ["{0:.2%}".format(value) for value in cf_mtx.flatten()/np.sum(cf_mtx)]
  box_labels = [f"{v1}\n({v2})" for v1, v2 in zip(group_counts, group_percentages)]
  box_labels = np.asarray(box_labels).reshape(6, 6)

  plt.figure(figsize = (12, 10))
  sns.heatmap(cf_mtx, xticklabels=base.class_indices.values(), yticklabels=base.class_indices.values(),
            cmap="YlGnBu", fmt="", annot=box_labels)
  plt.xlabel('Predicted Classes')
  plt.ylabel('True Classes')
  plt.show()



---



# Avaliação Holdout

In [ ]:
img_shape = (300, 300, 3)
base_directory = '/content/drive/MyDrive/IC/casting_data/casting_data'

train_path = base_directory + '/train'
val_path = base_directory + '/test'

batch_size = 256


gen_train = ImageDataGenerator(rescale=1./255,
                               horizontal_flip=True,
                               vertical_flip=True,
                               zoom_range=0.1,
                               brightness_range=[0.9,1.0])

base_train = gen_train.flow_from_directory(train_path,
                                           target_size=(300, 300),
                                           batch_size=batch_size,
                                           class_mode='binary',
                                           shuffle=False)

gen_val = ImageDataGenerator(rescale=1./255)

base_val = gen_val.flow_from_directory(val_path,
                                         target_size=(300, 300),
                                         batch_size=batch_size,
                                         class_mode='binary',
                                         shuffle=False)

In [ ]:
def create_rede():
  rede_neural = Sequential()

  rede_neural.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=img_shape, activation='relu', padding='same'))
  rede_neural.add(MaxPooling2D(pool_size=(2,2), strides=2))
  rede_neural.add(BatchNormalization())

  rede_neural.add(Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same'))
  rede_neural.add(MaxPooling2D(pool_size=(2,2), strides=2))
  rede_neural.add(BatchNormalization())

  rede_neural.add(Flatten(input_shape=img_shape))

  rede_neural.add(Dense(units=64, activation='relu'))
  rede_neural.add(Dropout(rate=0.3))
  rede_neural.add(Dense(units=1, activation='sigmoid'))

  rede_neural.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics = ['binary_accuracy'])

  return rede_neural


def create_mobile():
  base_model = MobileNetV2(input_shape=img_shape,
                          include_top=False,
                          weights='imagenet')

  base_model.trainable = True

  fine_tune_at = 153

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  inputs = tf.keras.Input(shape=img_shape)
  x = base_model(inputs, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  outputs = prediction_layer(x)
  model_mobile = tf.keras.Model(inputs, outputs)

  model_mobile.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics = ['binary_accuracy'])

  return model_mobile

def create_vgg():
  base_model = VGG16(input_shape=img_shape,
                         include_top=False,
                         weights='imagenet')

  base_model.trainable = True

  fine_tune_at = 18

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  inputs = tf.keras.Input(shape=img_shape)
  x = base_model(inputs, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  outputs = prediction_layer(x)
  model_vgg = tf.keras.Model(inputs, outputs)

  model_vgg.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics = ['binary_accuracy'])

  return model_vgg

def create_xception():
  base_model = Xception(input_shape=img_shape,
                         include_top=False,
                         weights='imagenet')

  base_model.trainable = True

  fine_tune_at = 130

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  inputs = tf.keras.Input(shape=img_shape)
  x = base_model(inputs, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  outputs = prediction_layer(x)
  model_xception = tf.keras.Model(inputs, outputs)

  model_xception.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics = ['binary_accuracy'])

  return model_xception

def create_inception():
  base_model = InceptionV3(input_shape=img_shape,
                         include_top=False,
                         weights='imagenet')

  base_model.trainable = True

  fine_tune_at = 309

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  inputs = tf.keras.Input(shape=img_shape)
  x = base_model(inputs, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  outputs = prediction_layer(x)
  model_inception = tf.keras.Model(inputs, outputs)

  model_inception.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics = ['binary_accuracy'])

  return model_inception

In [ ]:
dict = {
    "Rede Neural" : create_rede,
    "MobileNetV2": create_mobile,
    "ResNet50" : create_resnet,
    "VGG16" : create_vgg,
    "Xception" : create_xception,
    "InceptionV3" : create_inception,
}

models = ["Rede Neural", "MobileNetV2", "ResNet50", "VGG16", "Xception", "InceptionV3"]

for model_name in models:
  model = dict[model_name]()

  history = model.fit(base_train, epochs=30, validation_data=val_test)

  grafico(history, model_name)

  evaluate(model, base_val, model_name)



---



# Grad-CAM

In [ ]:
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, base_model, model, last_conv_layer_name, classifier_layer_names):
    last_conv_layer = base_model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(base_model.inputs, last_conv_layer.output)

    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)

        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)

    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)

    jet = plt.colormaps["jet"]

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    superimposed_img.save(cam_path)

    display(Image(cam_path))

In [ ]:
for layer in model.layers:
    print(layer.name)

base_model = model.layers[1]

for layer in base_model.layers:
  print(layer.name)

last_conv_layer_name = 'block5_pool'
preprocess_input = keras.applications.mobilenet_v2.preprocess_input
decode_predictions = keras.applications.mobilenet_v2.decode_predictions

classifier_layer_names = [layer.name for layer in model.layers]
del classifier_layer_names[1]

print(classifier_layer_names)

heatmaps = []

for _ in range(base_test.__len__()):
  for img in base_test.next()[0]:
    np.insert(img, 0, None)
    heatmap = make_gradcam_heatmap(tf.expand_dims(img,axis=0), base_model, model, last_conv_layer_name, classifier_layer_names)
    heatmaps.append(heatmap)

# Display heatmap
plt.matshow(heatmaps[0])
plt.show()

dict = {
    "Ok": '/content/drive/MyDrive/IC/casting_with_val/test/ok_front/cast_ok_0_1213.jpeg',
    "Def": '/content/drive/MyDrive/IC/casting_with_val/test/def_front/cast_def_0_1153.jpeg'
}

images = ['Ok', 'Def']

for img, heatmap in zip(images, heatmaps):
  img_path = dict[img]
  print(img)
  display(Image(img_path))
  save_and_display_gradcam(img_path, heatmap)

for img, heatmap in zip(images, heatmaps):
  img_path = dict[img]
  print(img)
  display(Image(img_path))
  save_and_display_gradcam(img_path, heatmap)